In [8]:
import os
import numpy as np
import pandas as pd
import cv2
from pycocotools.coco import COCO
from pycocotools import mask as maskUtils

In [9]:
# Define directories
image_directory = '../data/deigo/annotate-frames'  # Original Images
depth_map_directory = '../data/deigo/depth-maps/annotate-frames-dmap'  # Depth Map Images
json_file = '../data/deigo/diego-pipe-annotations/annotations/instances_default.json'  # JSON file

# Update the path to save results
results_csv_path = '../data/deigo/diego.csv'

In [10]:
# Load the COCO annotations
coco = COCO(json_file)

# Create a mapping from category_id to category_name
cats = coco.loadCats(coco.getCatIds())
category_mapping = {cat['id']: cat['name'] for cat in cats}
print("Category mapping:")
print(category_mapping)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Category mapping:
{1: '50', 2: '60', 3: '70', 4: '80', 5: '90'}


In [11]:
def load_depth_map(img_name):
    # Construct the depth map filename
    depth_map_filename = f"{img_name}_pred04.png"  # img_name without extension + '_pred04.png'
    depth_map_path = os.path.join(depth_map_directory, depth_map_filename)
    
    # Check if the depth map exists
    if not os.path.exists(depth_map_path):
        print(f"Depth map for image {img_name} not found at {depth_map_path}.")
        return None
    
    # Load the depth map image
    depth_map = cv2.imread(depth_map_path, cv2.IMREAD_UNCHANGED)
    if depth_map is None:
        print(f"Failed to load depth map for image {img_name}.")
        return None
    
    # Normalize the depth map if necessary (adjust based on your depth map format)
    if depth_map.dtype == np.uint8:
        # 8-bit depth map
        depth_map_normalized = depth_map / 255.0
    elif depth_map.dtype == np.uint16:
        # 16-bit depth map
        depth_map_normalized = depth_map / 65535.0
    else:
        # If depth map is already in float format between 0 and 1
        depth_map_normalized = depth_map

    return depth_map_normalized

In [12]:
def create_mask_from_annotation(ann, img_height, img_width):
    segmentation = ann['segmentation']
    
    if isinstance(segmentation, list):  # Polygon format
        mask = np.zeros((img_height, img_width), dtype=np.uint8)
        # Convert polygons to a mask
        for poly in segmentation:
            poly = np.array(poly).reshape((-1, 2))  # Reshape to (N, 2)
            cv2.fillPoly(mask, [np.int32(poly)], 1)  # Fill the polygon with 1's

    elif isinstance(segmentation, dict) and 'counts' in segmentation:  # RLE format
        rle = segmentation
        if isinstance(rle['counts'], list):  # Uncompressed RLE
            rle = maskUtils.frPyObjects([rle], img_height, img_width)
        mask = maskUtils.decode(rle)  # Decode RLE to binary mask

    else:
        mask = None  # Segmentation format not recognized

    return mask

In [13]:
# List to store results
results = []

# Get all image IDs
image_ids = coco.getImgIds()
print(f"Total number of images: {len(image_ids)}")

Total number of images: 17


In [14]:
# Process each image
for img_id in image_ids:
    # Load image information
    img_info = coco.loadImgs(img_id)[0]
    img_name_with_ext = img_info['file_name']  # e.g., '1.png'
    img_name = os.path.splitext(img_name_with_ext)[0]  # e.g., '1'
    img_height = img_info['height']
    img_width = img_info['width']
    
    print(f"Processing image ID {img_id}, name {img_name_with_ext}")

    # Load depth map
    depth_map = load_depth_map(img_name)
    if depth_map is None:
        print(f"Skipping image {img_name_with_ext} due to missing depth map.")
        continue  # Skip if depth map is not available

    # Get annotation IDs for the image
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    
    print(f"Number of annotations for image {img_name_with_ext}: {len(anns)}")
    
    # Process each annotation
    for ann in anns:
        annotation_id = ann['id']
        category_id = ann['category_id']
        category_name = category_mapping.get(category_id, "Unknown")
        print(f"Processing annotation ID {annotation_id}, category ID {category_id}, category name {category_name}")
        
        # Create mask for the annotation
        mask = create_mask_from_annotation(ann, img_height, img_width)
        if mask is None:
            print(f"Failed to create mask for annotation {annotation_id}")
            continue  # Skip if mask couldn't be created
        
        # Apply the mask to the depth map to extract depth values
        depth_values = depth_map[mask == 1]
        mean_depth = depth_values.mean() if depth_values.size > 0 else 0
        normalized_depth = mean_depth  # Depth map is already normalized between 0 and 1
        
        # Convert normalized depth to metric depth
        zmin = 0.4  # Minimum depth in meters (adjust if necessary)
        zmax = 20.0  # Maximum depth in meters (adjust if necessary)
        value_metric = (normalized_depth * (zmax - zmin)) + zmin
        
        print(f"Annotation ID {annotation_id}: mean depth {mean_depth}, normalized depth {normalized_depth}, value_metric {value_metric}")
        
        # Append the result to the list
        results.append({
            'image_name': img_name_with_ext,
            'image_id': img_id,
            'annotation_id': annotation_id,
            'category_id': category_id,
            'category_name': category_name,
            'normalized_depth': normalized_depth,
            'z-min': zmin,
            'z-max': zmax,
            'value_metric': value_metric
        })

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Display the DataFrame
print("Results DataFrame:")
print(results_df.head())

# Save the results to a CSV file
results_df.to_csv(results_csv_path, index=False)
print(f"Results saved to {results_csv_path}")

Processing image ID 1, name 1.png
Number of annotations for image 1.png: 3
Processing annotation ID 1, category ID 2, category name 60
Annotation ID 1: mean depth 0.22650808957462773, normalized depth 0.22650808957462773, value_metric 4.8395585556627045
Processing annotation ID 2, category ID 1, category name 50
Annotation ID 2: mean depth 0.1950980392156863, normalized depth 0.1950980392156863, value_metric 4.223921568627452
Processing annotation ID 3, category ID 5, category name 90
Annotation ID 3: mean depth 0.281126938437441, normalized depth 0.281126938437441, value_metric 5.910087993373844
Processing image ID 2, name 10.png
Number of annotations for image 10.png: 3
Processing annotation ID 4, category ID 2, category name 60
Annotation ID 4: mean depth 0.6445383117786121, normalized depth 0.6445383117786121, value_metric 13.032950910860798
Processing annotation ID 5, category ID 5, category name 90
Annotation ID 5: mean depth 0.7180322323698537, normalized depth 0.718032232369853